In [ ]:
# Instalación de dependencias
!pip install gradio
!pip install transformers
!pip install torch
!pip install sentencepiece
!pip install tf-keras

import gradio as gr
from transformers import (
    pipeline,
    AutoModelForSequenceClassification,
    AutoTokenizer
)
import torch
import numpy as np

# Configurar el dispositivo
device = "cuda" if torch.cuda.is_available() else "cpu"

# === CONFIGURACIÓN DEL CHATBOT ===
chat_generator = pipeline(
    'text-generation',
    model='microsoft/DialoGPT-small',
    device=device
)

# === CONFIGURACIÓN DEL ANALIZADOR DE SENTIMIENTOS ===
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
sentiment_tokenizer = AutoTokenizer.from_pretrained(model_name)
sentiment_model = AutoModelForSequenceClassification.from_pretrained(model_name)
sentiment_model.to(device)

def chatbot(titulo):
    if not titulo.strip():
        return "Por favor, escribe un título.", None, None

    try:
        # Generar respuesta del chatbot con el título (por ejemplo, una frase de explicación sobre el título)
        respuesta = chat_generator(
            titulo,
            max_length=100,
            temperature=0.7,
            do_sample=True,
            top_p=0.95,
            num_return_sequences=1
        )[0]['generated_text']

        respuesta = respuesta.replace(titulo, "").strip()

        # Aquí evaluamos el "sentimiento" del título, aunque en este caso podríamos ajustar el análisis para evaluar su relevancia en un contexto económico
        inputs = sentiment_tokenizer(titulo, return_tensors="pt", truncation=True,
                                   max_length=512, padding=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        outputs = sentiment_model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        rating = torch.argmax(predictions).item() + 1
        confidence = predictions[0][rating-1].item()

        if rating < 2:
            sentimiento = "Despropiado"
        elif rating == 2:
            sentimiento = "Poco Apropiado"
        elif rating == 3:
            sentimiento = "Neutral"
        elif rating == 4:
            sentimiento = "Apropiado"
        else:
            sentimiento = "Muy Apropiado"

        sentimiento_completo = f"{sentimiento} ({rating} estrellas)"
        confianza = round(confidence * 100, 2)

        return respuesta, sentimiento_completo, confianza

    except Exception as e:
        return f"Error: {str(e)}", "Error en el análisis", 0.0

# Crear la interfaz
demo = gr.Interface(
    fn=chatbot,
    inputs=[
        gr.Textbox(
            placeholder="Escribe el título del paper...",
            label="Título del Paper",
            lines=2
        )
    ],
    outputs=[
        gr.Textbox(label="Explicación del Título (Generada por el Chatbot)"),
        gr.Label(label="Evaluación del Título"),
        gr.Number(label="Confianza del Análisis (%)")
    ],
    title="Análisis de Títulos de Papers Económicos",
    description="Este modelo analiza la relevancia de un título de paper en el campo económico y genera una breve explicación.",
    examples=[
        ["Impact of Inflation on Economic Growth in Developing Countries"],
        ["The Role of Microfinance in Alleviating Poverty in Rural Areas"],
        ["Global Trade Patterns and their Effect on Emerging Economies"]
    ],
    allow_flagging="never",
    cache_examples=True
)

# Lanzar la interfaz
demo.launch(share=True, debug=True)



[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.6/259.6 MB 1.7 MB/s eta 0:00:0000:0100:02m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 1.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 3.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: ml-dtypes
    Found existi

2024-11-25 12:19:58.732454: W external/local_tsl/tsl/lib/monitoring/collection_registry.cc:81] Trying to register 2 metrics with the same name: /tensorflow/api/resource_variables. The old value will be erased in order to register a new one. Please check if you link the metric more than once, or if the name is already used by other metrics.
Error while downloading from https://cdn-lfs.hf.co/microsoft/DialoGPT-small/ea50e39b0e9368b9110cc5ab48012ddbc7bd90f2b17410aed643a7404acac567?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1732814399&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMjgxNDM5OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9taWNyb3NvZnQvRGlhbG9HUFQtc21hbGwvZWE1MGUzOWIwZTkzNjhiOTExMGNjNWFiNDgwMTJkZGJjN2JkOTBmMmIxNzQxMGFlZDY0M2E3NDA0YWNhYzU2Nz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=uiLzqO2-NuOk8Bqq-FJHX56rf63EgJLPdeDzzbi%7EmHaKf1nms

* Running on local URL:  http://127.0.0.1:7860
Caching examples at: '/Users/esteban/Personal/Noveno Semestre/HE2 Inteligencia Artificial/Parcial-3/.gradio/cached_examples/18'
Caching example 1/5


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Caching example 2/5
Caching example 3/5
Caching example 4/5
Caching example 5/5


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


* Running on public URL: https://40ab4581906794ab95.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://40ab4581906794ab95.gradio.live
